In [24]:
import sys, os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
from scipy.spatial.transform import Rotation
import pytorch3d


from human_body_prior.tools.omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

In [25]:
data_root = '/media/erik/DATA'
# raw_data_dir = f'{data_root}/behave_raw'
# stage_1_output = f'{data_root}/behave_processed/pose_data_behave'
# stage_2_output = f'{data_root}/behave_processed/joints_behave'

raw_data_dir = f'{data_root}/grab/grab_preprocessed'
stage_1_output = f'{data_root}/grab/pose_data_grab'
stage_2_output = f'{data_root}/grab/joints_grab'

# raw_data_dir = '/media/erik/DATA/grab_preprocessed'
# stage_1_output = '/media/erik/DATA/pose_data_grab'
# stage_2_output = '/media/erik/DATA/joints_grab'

In [26]:
# comp_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# this needs some really high memory GPUs, so default to CPU
comp_device = torch.device("cpu")

print('Computation device:', comp_device)
# %%
from human_body_prior.body_model.body_model import BodyModel

male_bm_path = './body_models/smplh/male/model.npz'
male_dmpl_path = './body_models/dmpls/male/model.npz'

female_bm_path = './body_models/smplh/female/model.npz'
female_dmpl_path = './body_models/dmpls/female/model.npz'

num_betas = 10 # number of body parameters
num_dmpls = 8 # number of DMPL parameters

# male_bm = BodyModel(bm_fname=male_bm_path, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=male_dmpl_path).to(comp_device)
male_bm = BodyModel(bm_fname=male_bm_path, num_betas=num_betas).to(comp_device)
faces = c2c(male_bm.f)

# female_bm = BodyModel(bm_fname=female_bm_path, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=female_dmpl_path).to(comp_device)
female_bm = BodyModel(bm_fname=female_bm_path, num_betas=num_betas).to(comp_device)

paths = []
folders = []
dataset_names = []
for root, dirs, files in os.walk(raw_data_dir):
    folders.append(root)
    for name in files:
        dataset_name = root.split('/')[1]
        if dataset_name not in dataset_names:
            dataset_names.append(dataset_name)
        if 'smpl_fit_all.npz' in files:
            paths.append(os.path.join(root, 'smpl_fit_all.npz'))

Computation device: cpu


In [29]:
import pytorch3d.transforms


save_folders = [folder.replace(raw_data_dir, stage_1_output) for folder in folders]
for folder in save_folders:
    os.makedirs(folder, exist_ok=True)
group_path = [[path for path in paths if name in path] for name in dataset_names]

# %%
# trans_matrix = np.array([[1.0, 0.0, 0.0],
#                             [0.0, -1.0, 0.0],
#                             [0.0, 0.0, -1.0]])

trans_matrix = np.array([[1.0, 0.0, 0.0],
                            [0.0, 1.0, 0.0],
                            [0.0, 0.0, 1.0]])

ex_fps = 30
def amass_to_pose(src_path, save_path):
    seq_info_path = src_path.replace("smpl_fit_all.npz", "info.json")
    with open(seq_info_path, "r") as f:
        seq_info = json.load(f)
    gender = seq_info["gender"]

    src_path_obj = src_path.replace('smpl_fit_all.npz', 'object_fit_all.npz')

    bdata = np.load(src_path, allow_pickle=True)
    bdata_obj = np.load(src_path_obj, allow_pickle=True)

    # load hand data
    src_path_lhand = src_path.replace('smpl_fit_all.npz', 'lhand_data.npz')
    src_path_rhand = src_path.replace('smpl_fit_all.npz', 'rhand_data.npz')
    data_lhand = np.load(src_path_lhand, allow_pickle=True)
    data_rhand = np.load(src_path_rhand, allow_pickle=True)
    # bdata_lhand = bdata_lhand['joints'] # 1 x T x 21 x 3
    # bdata_rhand = bdata_rhand['joints'] # 1 x T x 21 x 3

    # bdata_lhand = torch.Tensor(bdata_lhand).to(comp_device).squeeze(0) # T x 21 x 3
    # bdata_rhand = torch.Tensor(bdata_rhand).to(comp_device).squeeze(0) # T x 21 x 3

    bdata_lhand = data_lhand['hand_pose'].squeeze(0) # T x 24
    bdata_rhand = data_rhand['hand_pose'].squeeze(0) # T x 24

    # print(f'hand data shape: {bdata_lhand.shape}')

    rot_axisang_lhand = np.array(data_lhand['global_orient'])
    rot_axisang_rhand = np.array(data_rhand['global_orient'])

    # TODO: fix this, the rotation is AXIS ANGLE, not euler angles, need to convert this approapriately on both ends of pipeline

    # convert to 6d representation
    # rot_lhand = Rotation.from_euler('xyz', rot_euler_lhand).as_matrix()
    # rot_rhand = Rotation.from_euler('xyz', rot_euler_rhand).as_matrix()

    rot_lhand = pytorch3d.transforms.axis_angle_to_matrix(torch.Tensor(rot_axisang_lhand).to(comp_device))
    rot_rhand = pytorch3d.transforms.axis_angle_to_matrix(torch.Tensor(rot_axisang_rhand).to(comp_device))

    # convert to continuous 6d representation
    rot_lhand = pytorch3d.transforms.matrix_to_rotation_6d(rot_lhand).squeeze(0)
    rot_rhand = pytorch3d.transforms.matrix_to_rotation_6d(rot_rhand).squeeze(0)

    bdata_lhand = np.concatenate([rot_lhand, bdata_lhand], axis=-1) # T x 30
    bdata_rhand = np.concatenate([rot_rhand, bdata_rhand], axis=-1) # T x 30
    
    assert bdata_lhand.shape[1] == 30 and bdata_rhand.shape[1] == 30


    fps = 120
    frame_number = bdata['trans'].shape[0]

    fId = 0 # frame id of the mocap sequence
    pose_seq = []
    if gender == 'male':
        bm = male_bm
    else:
        bm = female_bm
    down_sample = int(fps / ex_fps)

    with torch.no_grad():
        root_orient = torch.Tensor(bdata['poses'][::down_sample, :3]).to(comp_device) # controls the global root orientation 

        pose_body = torch.Tensor(bdata['poses'][::down_sample, 3:66]).to(comp_device) # controls the body
        pose_hand = torch.Tensor(bdata['poses'][::down_sample, 66:]).to(comp_device) # controls the finger articulation
        betas = torch.Tensor(bdata['betas'][::down_sample]).to(comp_device) # controls the body shape
        trans = torch.Tensor(bdata['trans'][::down_sample]).to(comp_device)    
        body = bm(pose_body=pose_body, pose_hand=pose_hand, betas=betas, root_orient=root_orient)
        joint_loc = body.Jtr + trans[:, None]
        pose_seq = joint_loc

    # pose_seq = torch.cat(pose_seq, dim=0)
    
    pose_seq_np = pose_seq.detach().cpu().numpy()
    pose_seq_np_n = np.dot(pose_seq_np, trans_matrix)
    np.save(save_path, pose_seq_np_n)

    # process obj pose data
    angle, trans = bdata_obj['angles'], bdata_obj['trans']
    rot = Rotation.from_rotvec(angle).as_matrix()
    mat = np.eye(4)[np.newaxis].repeat(rot.shape[0], axis=0)
    mat[:, :3, :3] = rot
    mat[:, :3, 3] = trans
    trans_matrix_eye4 = np.eye(4)[np.newaxis]
    trans_matrix_eye4[0, :3, :3] = trans_matrix
    mat = trans_matrix_eye4 @ mat

    rot, trans = mat[:, :3, :3], mat[:, :3, 3]
    rot = Rotation.from_matrix(rot).as_rotvec()
    obj_pose = np.concatenate([rot, trans], axis=-1)

    # downsample obj pose
    obj_pose = obj_pose[::down_sample]
    bdata_lhand = bdata_lhand[::down_sample]
    bdata_rhand = bdata_rhand[::down_sample]

    save_path_obj = save_path.replace('smpl_fit_all.npy', 'object_fit_all.npy')
    np.save(save_path_obj, obj_pose)

    save_path_lhand = save_path.replace('smpl_fit_all.npy', 'lhand_data.npy')
    np.save(save_path_lhand, bdata_lhand)

    save_path_rhand = save_path.replace('smpl_fit_all.npy', 'rhand_data.npy')
    np.save(save_path_rhand, bdata_rhand)
    
    return fps

In [30]:
group_path = group_path
all_count = sum([len(paths) for paths in group_path])
cur_count = 0

import time
for paths in group_path:
    dataset_name = paths[0].split('/')[2]
    pbar = tqdm(paths)
    pbar.set_description('Processing: %s'%dataset_name)
    fps = 0
    for path in pbar:
        save_path = path.replace(raw_data_dir, stage_1_output)
        save_path = save_path[:-3] + 'npy'
        fps = amass_to_pose(path, save_path)
        
    cur_count += len(paths)
    print('Processed / All (fps %d): %d/%d'% (fps, cur_count, all_count) )
    time.sleep(0.5)

Processing: erik:   0%|          | 1/9345 [00:00<22:48,  6.83it/s]

hand data shape: (960, 24)
hand data shape: (960, 24)


Processing: erik:   0%|          | 3/9345 [00:00<15:20, 10.15it/s]

hand data shape: (960, 24)
hand data shape: (960, 24)
hand data shape: (960, 24)


Processing: erik:   0%|          | 7/9345 [00:00<13:21, 11.65it/s]

...

hand data shape: (1471, 24)
hand data shape: (1471, 24)


Processing: erik:   0%|          | 26/9345 [00:03<21:17,  7.29it/s]

hand data shape: (1471, 24)
hand data shape: (1471, 24)


Processing: erik:   0%|          | 26/9345 [00:03<21:06,  7.36it/s]


KeyboardInterrupt: 

In [ ]:
import codecs as cs
import pandas as pd
import numpy as np
from tqdm import tqdm
from os.path import join as pjoin

def swap_left_right(data):
    assert len(data.shape) == 3 and data.shape[-1] == 3
    data = data.copy()
    data[..., 0] *= -1
    right_chain = [2, 5, 8, 11, 14, 17, 19, 21]
    left_chain = [1, 4, 7, 10, 13, 16, 18, 20]
    left_hand_chain = [22, 23, 24, 34, 35, 36, 25, 26, 27, 31, 32, 33, 28, 29, 30]
    right_hand_chain = [43, 44, 45, 46, 47, 48, 40, 41, 42, 37, 38, 39, 49, 50, 51]
    tmp = data[:, right_chain]
    data[:, right_chain] = data[:, left_chain]
    data[:, left_chain] = tmp
    if data.shape[1] > 24:
        tmp = data[:, right_hand_chain]
        data[:, right_hand_chain] = data[:, left_hand_chain]
        data[:, left_hand_chain] = tmp
    return data

total_amount = len(group_path[0])
fps = 30

for i in tqdm(range(total_amount)):
    path = group_path[0][i]
    source_path = path.replace(raw_data_dir, stage_1_output)
    source_path = source_path[:-3] + 'npy'
    try:
        data = np.load(source_path)
    except:
        print('Error: ', source_path)
        continue
    new_name = source_path.split('/')[-2]
    # data[..., 0] *= -1
    
    # data_m = swap_left_right(data)

    source_path_obj = source_path.replace('smpl_fit_all.npy', 'object_fit_all.npy')
    data_obj = np.load(source_path_obj)
    if not os.path.exists(stage_2_output):
        os.makedirs(stage_2_output, exist_ok=True)

    np.save(pjoin(stage_2_output, new_name), data)
    # np.save(pjoin(save_dir, 'M'+new_name), data_m)

IndexError: list index out of range